In [1]:
%run ../yp_utils.py

# Initial setup

In [2]:
paper_pmid = 33903247
paper_name = 'pallares_abergel_2021' 

In [3]:
datasets = pd.read_csv('extras/YeastPhenome_' + str(paper_pmid) + '_datasets_list.txt', sep='\t', header=None, names=['dataset_id', 'name'])

In [4]:
datasets.set_index('dataset_id', inplace=True)

# Load & process the data

In [5]:
original_data = pd.read_excel('raw_data/Overall Toxicogenomics Data - Lns.xlsx', sheet_name='Sheet1', skiprows=1)

In [6]:
print('Original data dimensions: %d x %d' % (original_data.shape))

Original data dimensions: 4291 x 15


In [7]:
original_data.head()

,ORF,Standard Name,Lanthanum,Praseodymium,Neodymium,Samarium,Europium,Gadolinium,Terbium,Dysprosium,Holmium,Erbium,Thulium,Ytterbium,Lutetium
0,YLR020C,YEH2,-4.900316,-0.113639,-0.252636,-0.057131,-4.528212,-1.482404,-2.000656,-3.478031,-7.648509,-6.353022,-2.108975,-3.005943,-0.945948
1,YMR183C,SSO2,-3.840186,0.296654,-0.418245,-0.469292,-7.581696,-5.654397,-3.531039,-4.604803,-6.899329,-2.691471,-1.188448,-2.246067,-0.570282
2,YDL052C,SLC1,-2.660804,-2.433811,-1.669029,-1.431186,-5.879309,-5.504203,-4.786134,-5.099304,-7.112161,2.003129,0.245958,-0.228923,-0.629999
3,YIL065C,FIS1,-0.262174,-1.203524,-0.367700,-0.395253,-5.571957,-2.852731,-3.294151,-3.222766,-7.456789,-2.494217,-1.899803,-2.303305,-1.440318
4,YIL077C,NaN,-0.319544,-0.935148,-0.669546,-0.962880,-4.390158,-1.664524,-3.161976,-3.210569,-6.778105,-3.371183,-2.015285,-1.814728,-1.206460


In [8]:
original_data['orf'] = original_data['ORF'].astype(str)

In [9]:
# Eliminate all white spaces & capitalize
original_data['orf'] = clean_orf(original_data['orf'])

In [10]:
# Translate to ORFs 
original_data['orf'] = translate_sc(original_data['orf'], to='orf')

In [11]:
# Make sure everything translated ok
t = looks_like_orf(original_data['orf'])
print(original_data.loc[~t,])

Empty DataFrame
Columns: [ORF, Standard Name, Lanthanum, Praseodymium, Neodymium, Samarium, Europium, Gadolinium, Terbium, Dysprosium, Holmium, Erbium, Thulium, Ytterbium, Lutetium, orf]
Index: []


In [17]:
original_data.drop(columns=['Standard Name','ORF'], inplace=True)

In [13]:
original_data.set_index('orf', inplace=True)

In [18]:
original_data.head()

,Lanthanum,Praseodymium,Neodymium,Samarium,Europium,Gadolinium,Terbium,Dysprosium,Holmium,Erbium,Thulium,Ytterbium,Lutetium
orf,,,,,,,,,,,,,
YLR020C,-4.900316,-0.113639,-0.252636,-0.057131,-4.528212,-1.482404,-2.000656,-3.478031,-7.648509,-6.353022,-2.108975,-3.005943,-0.945948
YMR183C,-3.840186,0.296654,-0.418245,-0.469292,-7.581696,-5.654397,-3.531039,-4.604803,-6.899329,-2.691471,-1.188448,-2.246067,-0.570282
YDL052C,-2.660804,-2.433811,-1.669029,-1.431186,-5.879309,-5.504203,-4.786134,-5.099304,-7.112161,2.003129,0.245958,-0.228923,-0.629999
YIL065C,-0.262174,-1.203524,-0.367700,-0.395253,-5.571957,-2.852731,-3.294151,-3.222766,-7.456789,-2.494217,-1.899803,-2.303305,-1.440318
YIL077C,-0.319544,-0.935148,-0.669546,-0.962880,-4.390158,-1.664524,-3.161976,-3.210569,-6.778105,-3.371183,-2.015285,-1.814728,-1.206460


In [19]:
original_data = original_data.apply(pd.to_numeric, axis=1)

In [20]:
original_data = original_data.groupby(original_data.index).mean()

In [21]:
original_data.shape

(4270, 13)

# Prepare the final dataset

In [22]:
data = original_data.copy()

In [23]:
dataset_ids = [21906, 21908, 21909, 21910, 21911, 21912, 21918, 21913, 21914, 21919, 21915, 21916, 21917]
datasets = datasets.reindex(index=dataset_ids)

In [24]:
lst = [datasets.index.values, ['value']*datasets.shape[0]]
tuples = list(zip(*lst))
idx = pd.MultiIndex.from_tuples(tuples, names=['dataset_id','data_type'])
data.columns = idx

In [25]:
data.head()

dataset_id,21906,21908,21909,21910,21911,21912,21918,21913,21914,21919,21915,21916,21917
data_type,value,value,value,value,value,value,value,value,value,value,value,value,value
orf,,,,,,,,,,,,,
YAL001C,0.493615,-0.030919,-0.849198,0.758048,-2.161403,-0.878903,-0.878673,-3.797008,3.278457,NaN,1.261723,3.098041,NaN
YAL002W,-0.086373,0.477618,0.849104,0.412835,-4.166589,-1.186373,-0.221337,-1.124700,3.343794,NaN,2.327422,0.173197,NaN
YAL004W,-0.082562,-0.016978,-0.235208,-0.398059,0.185748,-0.315955,-0.016599,0.539963,NaN,NaN,-0.139995,-0.523985,NaN
YAL005C,0.115399,0.184295,0.237727,0.163313,1.692353,0.256507,0.103818,0.513941,1.367297,NaN,0.181848,-0.212665,NaN
YAL007C,0.593529,-0.352209,0.252640,-0.120375,0.072241,-0.911172,0.434622,0.443389,1.159416,NaN,0.630518,0.796958,NaN


## Subset to the genes currently in SGD

In [26]:
genes = pd.read_csv(path_to_genes, sep='\t', index_col='id')
genes = genes.reset_index().set_index('systematic_name')
gene_ids = genes.reindex(index=data.index.values)['id'].values
num_missing = np.sum(np.isnan(gene_ids))
print('ORFs missing from SGD: %d' % num_missing)

ORFs missing from SGD: 21


In [27]:
data['gene_id'] = gene_ids
data = data.loc[data['gene_id'].notnull()]
data['gene_id'] = data['gene_id'].astype(int)
data = data.reset_index().set_index(['gene_id','orf'])

data.head()

,dataset_id,21906,21908,21909,21910,21911,21912,21918,21913,21914,21919,21915,21916,21917
,data_type,value,value,value,value,value,value,value,value,value,value,value,value,value
gene_id,orf,,,,,,,,,,,,,
1,YAL001C,0.493615,-0.030919,-0.849198,0.758048,-2.161403,-0.878903,-0.878673,-3.797008,3.278457,NaN,1.261723,3.098041,NaN
2,YAL002W,-0.086373,0.477618,0.849104,0.412835,-4.166589,-1.186373,-0.221337,-1.124700,3.343794,NaN,2.327422,0.173197,NaN
1863,YAL004W,-0.082562,-0.016978,-0.235208,-0.398059,0.185748,-0.315955,-0.016599,0.539963,NaN,NaN,-0.139995,-0.523985,NaN
4,YAL005C,0.115399,0.184295,0.237727,0.163313,1.692353,0.256507,0.103818,0.513941,1.367297,NaN,0.181848,-0.212665,NaN
5,YAL007C,0.593529,-0.352209,0.252640,-0.120375,0.072241,-0.911172,0.434622,0.443389,1.159416,NaN,0.630518,0.796958,NaN


# Normalize

In [28]:
data_norm = normalize_phenotypic_scores(data, has_tested=True)

In [29]:
# Assign proper column names
lst = [datasets.index.values, ['valuez']*datasets.shape[0]]
tuples = list(zip(*lst))
idx = pd.MultiIndex.from_tuples(tuples, names=['dataset_id','data_type'])
data_norm.columns = idx

In [30]:
data_norm[data.isnull()] = np.nan
data_all = data.join(data_norm)

data_all.head()

,dataset_id,21906,21908,21909,21910,21911,21912,21918,21913,21914,21919,...,21910,21911,21912,21918,21913,21914,21919,21915,21916,21917
,data_type,value,value,value,value,value,value,value,value,value,value,...,valuez,valuez,valuez,valuez,valuez,valuez,valuez,valuez,valuez,valuez
gene_id,orf,,,,,,,,,,,,,,,,,,,,,
1,YAL001C,0.493615,-0.030919,-0.849198,0.758048,-2.161403,-0.878903,-0.878673,-3.797008,3.278457,NaN,...,1.610195,-1.007639,-0.688999,-0.992028,-3.077441,1.670532,NaN,1.149532,2.194109,NaN
2,YAL002W,-0.086373,0.477618,0.849104,0.412835,-4.166589,-1.186373,-0.221337,-1.124700,3.343794,NaN,...,0.858236,-1.767262,-1.046228,-0.288722,-0.954995,1.703637,NaN,2.179173,0.068331,NaN
1863,YAL004W,-0.082562,-0.016978,-0.235208,-0.398059,0.185748,-0.315955,-0.016599,0.539963,NaN,NaN,...,-0.908087,-0.118470,-0.034945,-0.069666,0.367142,NaN,NaN,-0.204759,-0.438382,NaN
4,YAL005C,0.115399,0.184295,0.237727,0.163313,1.692353,0.256507,0.103818,0.513941,1.367297,NaN,...,0.314718,0.452275,0.630161,0.059172,0.346474,0.702180,NaN,0.106194,-0.212114,NaN
5,YAL007C,0.593529,-0.352209,0.252640,-0.120375,0.072241,-0.911172,0.434622,0.443389,1.159416,NaN,...,-0.303224,-0.161470,-0.726490,0.413110,0.290439,0.596850,NaN,0.539684,0.521681,NaN


# Print out

In [31]:
for f in ['value','valuez']:
    df = data_all.xs(f, level='data_type', axis=1).copy()
    df.columns = datasets['name'].values
    df = df.droplevel('gene_id', axis=0)
    df.to_csv(paper_name + '_' + f + '.txt', sep='\t')

# Save to DB

In [32]:
from IO.save_data_to_db3 import *

In [33]:
save_data_to_db(data_all, paper_pmid)

  0%|          | 0/13 [00:00<?, ?it/s]

Deleting all datasets for PMID 33903247...
Inserting the new data...


100%|██████████| 13/13 [01:12<00:00,  5.59s/it]

Updating the data_modified_on field...
